# LightGCN - simplified GCN model for recommendation

This notebook serves as an introduction to LightGCN [1], which is an simple, linear and neat Graph Convolution Network (GCN) [3] model for recommendation.

## 0 Global Settings and Imports

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:51:59) 
[Clang 14.0.6 ]
Pandas version: 1.5.3
Tensorflow version: 2.9.1


In [2]:
DATA_FILE_NAME = "../Data/20230721T041206_sales_2023_basic_single_events_removed.csv"
THIS_ENGINE_NAME = "lightgcn_field_3"

# country
COUNTRY = "nigeria"

# top k items to recommend, for train & test
TOP_K_SPLIT_TRAIN_TEST = 10
# top k items to recommend, for final product recommendation output
TOP_K_WHOLE = 100
EXTRA_COLS = False

# fraction of location_skus to include in training dataset
TRAIN_FRAC = 0.75

# Model parameters
EPOCHS = 75
BATCH_SIZE = 1024
N_LAYERS = 3
LEARNING_RATE = 0.005
EVAL_EPOCH = 5

SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = THIS_ENGINE_NAME + "_hparams.yaml"
# user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
# item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

## 1 Data

### 1.1 Load and split data

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because SAR (typo?) generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.

In [3]:
df_all_cols = pd.read_csv(DATA_FILE_NAME)
df_all_cols = df_all_cols[df_all_cols["country"] == COUNTRY]

df = df_all_cols[["location_id", "product", "sl_sold"]]
df.rename(columns = {"location_id": "userID", "product": "itemID", "sl_sold": "rating"}, inplace = True)

df.head()

/var/folders/pz/g0trckmn61v44xkm8xvfx0n00000gn/T/ipykernel_58194/3279382176.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {"location_id": "userID", "product": "itemID", "sl_sold": "rating"}, inplace = True)


,userID,itemID,rating
509,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Mixanal Tablet,1
510,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Emzor Paracetamol 500mg Tablets x96,3
511,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Syrup Paracetamol 125mg/5ml (Emzor),20
512,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Em-Vit-C 100ml Syrup,2
513,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Funbact A 30g Cream,2


In [4]:
train, test = python_stratified_split(df, ratio = TRAIN_FRAC, seed = SEED)

### 1.2 Process data

`ImplicitCF` is a class that intializes and loads data for the training process. During the initialization of this class, user IDs and item IDs are reindexed, ratings greater than zero are converted into implicit positive interaction, and adjacency matrix $R$ of user-item graph is created. Some important methods of `ImplicitCF` are:

`get_norm_adj_mat`, load normalized adjacency matrix of user-item graph if it already exists in `adj_dir`, otherwise call `create_norm_adj_mat` to create the matrix and save the matrix if `adj_dir` is not `None`. This method will be called during the initialization process of LightGCN model.

`create_norm_adj_mat`, create normalized adjacency matrix of user-item graph by calculating $D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$, where $\mathbf{A}=\left(\begin{array}{cc}\mathbf{0} & \mathbf{R} \\ \mathbf{R}^{T} & \mathbf{0}\end{array}\right)$.

`train_loader`, generate a batch of training data — sample a batch of users and then sample one positive item and one negative item for each user. This method will be called before each epoch of the training process.


In [5]:
data = ImplicitCF(train=train, test=test, seed=SEED)

/Users/DanielShen/opt/anaconda3/envs/microsoft_rec/lib/python3.8/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


### 1.3 Prepare hyper-parameters

Important parameters of `LightGCN` model are:

`data`, initialized LightGCNDataset object.

`epochs`, number of epochs for training.

`n_layers`, number of layers of the model.

`eval_epoch`, if it is not None, evaluation metrics will be calculated on test set every "eval_epoch" epochs. In this way, we can observe the effect of the model during the training process.

`top_k`, the number of items to be recommended for each user when calculating ranking metrics.

A complete list of parameters can be found in `yaml_file`. We use `prepare_hparams` to read the yaml file and prepare a full set of parameters for the model. Parameters passed as the function's parameters will overwrite yaml settings.

In [6]:
hparams = prepare_hparams(yaml_file,
                          n_layers=N_LAYERS,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=LEARNING_RATE,
                          eval_epoch=EVAL_EPOCH,
                          top_k=TOP_K_SPLIT_TRAIN_TEST,
                         )

## 2 Train model

With data and parameters prepared, we can create the LightGCN model.

To train the model, we simply need to call the `fit()` method.

In [7]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


2023-08-02 13:57:51.805870: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 13:57:51.855613: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [8]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.2s: train loss = 0.62590 = (mf)0.62584 + (embed)0.00007
Epoch 2 (train)1.0s: train loss = 0.38232 = (mf)0.38193 + (embed)0.00038
Epoch 3 (train)1.1s: train loss = 0.33318 = (mf)0.33258 + (embed)0.00060
Epoch 4 (train)1.1s: train loss = 0.30885 = (mf)0.30817 + (embed)0.00069
Epoch 5 (train)1.1s + (eval)0.2s: train loss = 0.27948 = (mf)0.27867 + (embed)0.00081, recall = 0.07149, ndcg = 0.23194, precision = 0.22443, map = 0.03657
Epoch 6 (train)1.3s: train loss = 0.26046 = (mf)0.25951 + (embed)0.00095
Epoch 7 (train)1.0s: train loss = 0.25233 = (mf)0.25126 + (embed)0.00107
Epoch 8 (train)1.0s: train loss = 0.24182 = (mf)0.24063 + (embed)0.00119
Epoch 9 (train)1.1s: train loss = 0.23063 = (mf)0.22932 + (embed)0.00131
Epoch 10 (train)1.1s + (eval)0.3s: train loss = 0.22503 = (mf)0.22361 + (embed)0.00142, recall = 0.07670, ndcg = 0.24541, precision = 0.23652, map = 0.03933
Epoch 11 (train)1.1s: train loss = 0.21280 = (mf)0.21126 + (embed)0.00154
Epoch 12 (train)1.1s: train l

## 3 Prediction/Recommendation

Recommendation and evaluation have been performed on the specified test set during training. After training, we can also use the model to perform recommendation and evalution on other data. Here we still use `test` as test data, but `test` can be replaced by other data with similar data structure.

We can call `recommend_k_items` to recommend k items for each user passed in this function. We set `remove_seen=True` to remove the items already seen by the user. The function returns a dataframe, containing each user and top k items recommended to them and the corresponding ranking scores.

In [9]:
with Timer() as test_time:
    topk_scores = model.recommend_k_items(test, top_k=TOP_K_SPLIT_TRAIN_TEST, remove_seen=True)
print("Took {} seconds for prediction.".format(test_time.interval))

topk_scores.head()

Took 0.029627250000004324 seconds for training.


,userID,itemID,prediction
0,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Calcimax Tablets x30,10.448770
1,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Kiss Condoms,9.648853
2,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Ibucap x20,9.534927
3,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Danacid Tablets x96,9.446033
4,"""01aa6102-a054-4f25-a747-a39a4ea86769""",Tab Boska x20,9.389656


## 4 Evaluation

With `topk_scores` predicted by the model, we can evaluate how LightGCN performs on this test set.

In [10]:
eval_map = map_at_k(test, topk_scores, k=TOP_K_SPLIT_TRAIN_TEST)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K_SPLIT_TRAIN_TEST)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K_SPLIT_TRAIN_TEST)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K_SPLIT_TRAIN_TEST)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.047492
NDCG:	0.267060
Precision@K:	0.254156
Recall@K:	0.089995


## [Skipped] Infer embeddings

With `infer_embedding` method of LightGCN model, we can export the embeddings of users and items in the training set to CSV files for future use.

In [11]:
# model.infer_embedding(user_file, item_file)

## 5 Additional exploration

### Revised/copied code from packages

In [12]:
import numpy as np


from recommenders.utils.constants import (
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_RATING_COL,
    DEFAULT_PREDICTION_COL,
    DEFAULT_RELEVANCE_COL,
    DEFAULT_SIMILARITY_COL,
    DEFAULT_ITEM_FEATURES_COL,
    DEFAULT_ITEM_SIM_MEASURE,
    DEFAULT_K,
    DEFAULT_THRESHOLD,
)


def _get_rating_column(relevancy_method: str, **kwargs) -> str:
    r"""Helper utility to simplify the arguments of eval metrics
    Attemtps to address https://github.com/microsoft/recommenders/issues/1737.

    Args:
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.

    Returns:
        str: rating column name.
    """
    if relevancy_method != "top_k":
        if "col_rating" not in kwargs:
            raise ValueError("Expected an argument `col_rating` but wasn't found.")
        col_rating = kwargs.get("col_rating")
    else:
        col_rating = kwargs.get("col_rating", DEFAULT_RATING_COL)
    return col_rating


def merge_ranking_true_pred(
    rating_true,
    rating_pred,
    col_user,
    col_item,
    col_rating,
    col_prediction,
    relevancy_method,
    k=DEFAULT_K,
    threshold=DEFAULT_THRESHOLD,
):
    """Filter truth and prediction data frames on common users

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user (optional)
        threshold (float): threshold of top items per user (optional)

    Returns:
        pandas.DataFrame, pandas.DataFrame, int: DataFrame of recommendation hits, sorted by `col_user` and `rank`
        DataFrame of hit counts vs actual relevant items per user number of unique user ids
    """

    # Make sure the prediction and true data frames have the same set of users
    common_users = set(rating_true[col_user]).intersection(set(rating_pred[col_user]))
    rating_true_common = rating_true[rating_true[col_user].isin(common_users)]
    rating_pred_common = rating_pred[rating_pred[col_user].isin(common_users)]
    n_users = len(common_users)

    # Return hit items in prediction data frame with ranking information. This is used for calculating NDCG and MAP.
    # Use first to generate unique ranking values for each item. This is to align with the implementation in
    # Spark evaluation metrics, where index of each recommended items (the indices are unique to items) is used
    # to calculate penalized precision of the ordered items.
    if relevancy_method == "top_k":
        top_k = k
    elif relevancy_method == "by_threshold":
        top_k = threshold
    elif relevancy_method is None:
        top_k = None
    else:
        raise NotImplementedError("Invalid relevancy_method")
    df_hit = get_top_k_items(
        dataframe=rating_pred_common,
        col_user=col_user,
        col_rating=col_prediction,
        k=top_k,
    )
    df_hit = pd.merge(df_hit, rating_true_common, on=[col_user, col_item])[
        [col_user, col_item, "rank"]
    ]

    # count the number of hits vs actual relevant items per user
    df_hit_count = pd.merge(
        df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
        rating_true_common.groupby(col_user, as_index=False)[col_user].agg(
            {"actual": "count"}
        ),
        on=col_user,
    )

    return df_hit, df_hit_count, n_users


def get_top_k_items(
    dataframe, col_user=DEFAULT_USER_COL, col_rating=DEFAULT_RATING_COL, k=DEFAULT_K
):
    """Get the input customer-item-rating tuple in the format of Pandas
    DataFrame, output a Pandas DataFrame in the dense format of top k items
    for each user.

    Note:
        If it is implicit rating, just append a column of constants to be
        ratings.

    Args:
        dataframe (pandas.DataFrame): DataFrame of rating data (in the format
        customerID-itemID-rating)
        col_user (str): column name for user
        col_rating (str): column name for rating
        k (int or None): number of items for each user; None means that the input has already been
        filtered out top k items and sorted by ratings and there is no need to do that again.

    Returns:
        pandas.DataFrame: DataFrame of top k items for each user, sorted by `col_user` and `rank`
    """
    # Sort dataframe by col_user and (top k) col_rating
    if k is None:
        top_k_items = dataframe
    else:
        top_k_items = (
            dataframe.sort_values([col_user, col_rating], ascending=[True, False])
            .groupby(col_user, as_index=False)
            .head(k)
            .reset_index(drop=True)
        )
    # Add ranks
    top_k_items["rank"] = top_k_items.groupby(col_user, sort=False).cumcount() + 1
    return top_k_items


def ndcg_at_k_hist(
    rating_true,
    rating_pred,
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    col_prediction=DEFAULT_PREDICTION_COL,
    relevancy_method="top_k",
    k=DEFAULT_K,
    threshold=DEFAULT_THRESHOLD,
    score_type="binary",
    discfun_type="loge",
    **kwargs
):
    """Normalized Discounted Cumulative Gain (nDCG).

    Info: https://en.wikipedia.org/wiki/Discounted_cumulative_gain

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user
        threshold (float): threshold of top items per user (optional)
        score_type (str): type of relevance scores ['binary', 'raw', 'exp']. With the default option 'binary', the
            relevance score is reduced to either 1 (hit) or 0 (miss). Option 'raw' uses the raw relevance score.
            Option 'exp' uses (2 ** RAW_RELEVANCE - 1) as the relevance score
        discfun_type (str): type of discount function ['loge', 'log2'] used to calculate DCG.

    Returns:
        float: nDCG at k (min=0, max=1).
    """
    col_rating = _get_rating_column(relevancy_method, **kwargs)
    df_hit, _, _ = merge_ranking_true_pred(
        rating_true=rating_true,
        rating_pred=rating_pred,
        col_user=col_user,
        col_item=col_item,
        col_rating=col_rating,
        col_prediction=col_prediction,
        relevancy_method=relevancy_method,
        k=k,
        threshold=threshold,
    )

    if df_hit.shape[0] == 0:
        return 0.0

    df_dcg = df_hit.merge(rating_pred, on=[col_user, col_item]).merge(
        rating_true, on=[col_user, col_item], how="outer", suffixes=("_left", None)
    )

    if score_type == "binary":
        df_dcg["rel"] = 1
    elif score_type == "raw":
        df_dcg["rel"] = df_dcg[col_rating]
    elif score_type == "exp":
        df_dcg["rel"] = 2 ** df_dcg[col_rating] - 1
    else:
        raise ValueError("score_type must be one of 'binary', 'raw', 'exp'")

    if discfun_type == "loge":
        discfun = np.log
    elif discfun_type == "log2":
        discfun = np.log2
    else:
        raise ValueError("discfun_type must be one of 'loge', 'log2'")

    # Calculate the actual discounted gain for each record
    df_dcg["dcg"] = df_dcg["rel"] / discfun(1 + df_dcg["rank"])

    # Calculate the ideal discounted gain for each record
    df_idcg = df_dcg.sort_values([col_user, col_rating], ascending=False)
    df_idcg["irank"] = df_idcg.groupby(col_user, as_index=False, sort=False)[
        col_rating
    ].rank("first", ascending=False)
    df_idcg["idcg"] = df_idcg["rel"] / discfun(1 + df_idcg["irank"])

    # Calculate the actual DCG for each user
    df_user = df_dcg.groupby(col_user, as_index=False, sort=False).agg({"dcg": "sum"})

    # Calculate the ideal DCG for each user
    df_user = df_user.merge(
        df_idcg.groupby(col_user, as_index=False, sort=False)
        .head(k)
        .groupby(col_user, as_index=False, sort=False)
        .agg({"idcg": "sum"}),
        on=col_user,
    )

    # DCG over IDCG is the normalized DCG
    df_user["ndcg"] = df_user["dcg"] / df_user["idcg"]
    return df_user

### nDCG per location histogram

In [13]:
# import matplotlib.pyplot as plt
# from matplotlib.ticker import PercentFormatter

# count_not_percentage = False
# eval_with_ndcg_hist = ndcg_at_k_hist(test, all_predictions, col_user='location_id', col_item='product', col_rating='sl_sold', col_prediction='prediction', k=TOP_K_SPLIT_TRAIN_TEST)
# eval_ndcg_hist = eval_with_ndcg_hist["ndcg"]

# if count_not_percentage:
#     plt.hist(eval_ndcg_hist, bins = np.linspace(0, 1, 11))
#     plt.ylabel("Frequency")
# else:
#     plt.hist(eval_ndcg_hist, bins = np.linspace(0, 1, 11),\
#          weights = np.ones(len(eval_ndcg_hist)) / len(eval_ndcg_hist))
#     plt.ylabel("Percentage in bin")
#     plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
# plt.title("nDCG per location")
# plt.xlabel("nDCG value")
# plt.show()
# print("Average nDCG over all locations: " + str(eval_ndcg_hist.mean()))

### compute some stats

In [14]:
# n_locs_w_prod = {prod: 0 for prod in df_all_cols["product"]}
# n_prods_at_loc = {loc: 0 for loc in df_all_cols["location_id"]}
# membership = {}

# for _, row in df_all_cols.iterrows():
#     n_locs_w_prod[row["product"]] += 1
#     n_prods_at_loc[row["location_id"]] += 1
#     membership[row["location_id"]] = row["membership_type"]

# n_prods_at_loc_basic = {loc: n_prods for loc, n_prods in n_prods_at_loc.items() if membership[loc] == "basic"}

# n_locs = len(n_prods_at_loc)
# n_basic_locs = len(n_prods_at_loc_basic)

# print("Number of locations: " + str(n_locs))
# print("Number of basic locations: " + str(n_basic_locs))

### nDCG vs # products at location

In [15]:
# eval_loc_ndcg_hist = eval_with_ndcg_hist[["location_id", "ndcg"]]
# eval_ndcg_hist_n_prods = eval_loc_ndcg_hist.join(pd.Series(n_prods_at_loc).to_frame("n_prods"), on = "location_id", how = "left")

# eval_ndcg_hist_n_prods.plot(x = "n_prods", y = "ndcg", style = "o", legend = None)
# plt.title("nDCG vs. # products for all locations")
# plt.xlabel("# products")
# plt.ylabel("nDCG")
# plt.show()

### nDCG per location by number of products histogram

In [16]:
# min_n_prods_range = (0, 50, 100, 150, 200, 300, 400)
# max_n_prods_range = (50, 100, 150, 200, 300, 400, 500)
# count_not_percentage = False

# for (min_n_prods, max_n_prods) in zip(min_n_prods_range, max_n_prods_range):
#     eval_ndcg_hist_bounded_n_prods = eval_ndcg_hist_n_prods[(eval_ndcg_hist_n_prods["n_prods"] >= min_n_prods) & (eval_ndcg_hist_n_prods["n_prods"] < max_n_prods)]
#     eval_ndcg_hist_bounded_n_prods_ndcg_only = eval_ndcg_hist_bounded_n_prods["ndcg"]

#     if count_not_percentage:
#         plt.hist(eval_ndcg_hist_bounded_n_prods_ndcg_only, bins = np.linspace(0, 1, 11))
#         plt.ylabel("Frequency")
#     else:
#         plt.hist(eval_ndcg_hist_bounded_n_prods_ndcg_only, bins = np.linspace(0, 1, 11),\
#             weights = np.ones(len(eval_ndcg_hist_bounded_n_prods_ndcg_only)) / len(eval_ndcg_hist_bounded_n_prods_ndcg_only))
#         plt.ylabel("Percentage in bin")
#         plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
#     plt.title("nDCG per location with # products in [" + str(min_n_prods) + ", " + str(max_n_prods) + ")")
#     plt.xlabel("nDCG value")
#     plt.show()
#     print("Average nDCG for locations with # products in [" + str(min_n_prods) + ", " + str(max_n_prods) + "): " + str(eval_ndcg_hist_bounded_n_prods_ndcg_only.mean()))

## 6 Train, Predict, and Evaluate on Whole Dataset

Earlier, we had split train (for model training) and test (for evaluation). In implementation, we have train = whole dataset, and we can evaluate on test = whole dataset.

In [17]:
#TOP_K_WHOLE = 10
#EXTRA_COLS = True

## Data
data_whole = ImplicitCF(train=df, test=df, seed=SEED)

## Hyperparameters
hparams = prepare_hparams(yaml_file,
                          n_layers=N_LAYERS,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=LEARNING_RATE,
                          eval_epoch=EVAL_EPOCH,
                          top_k=TOP_K_WHOLE,
                         )

## Train
model_whole = LightGCN(hparams, data, seed=SEED)
with Timer() as train_time:
    model_whole.fit()
print("Took {} seconds for training.".format(train_time.interval))

## Predict
with Timer() as test_time:
    topk_scores_whole = model_whole.recommend_k_items(df, top_k=TOP_K_WHOLE, remove_seen=False)
print("Took {} seconds for prediction.".format(test_time.interval))

## Evaluate
eval_map_whole = map_at_k(df, topk_scores_whole, k=TOP_K_WHOLE)
eval_ndcg_whole = ndcg_at_k(df, topk_scores_whole, k=TOP_K_WHOLE)
eval_precision_whole = precision_at_k(df, topk_scores_whole, k=TOP_K_WHOLE)
eval_recall_whole = recall_at_k(df, topk_scores_whole, k=TOP_K_WHOLE)

print()
print("MAP:\t%f" % eval_map_whole,
      "NDCG:\t%f" % eval_ndcg_whole,
      "Precision@K:\t%f" % eval_precision_whole,
      "Recall@K:\t%f" % eval_recall_whole, sep='\n')

## Save recommendations
top_k_predictions_whole = get_top_k_items(topk_scores_whole, col_rating='prediction', k=TOP_K_WHOLE) # this adds a rank column
top_k_predictions_whole.drop("prediction", axis = 1, inplace = True)
# add column of true sales and rank of predicted products
top_all_true = get_top_k_items(df, k=df["itemID"].nunique())
top_k_predictions_whole = top_k_predictions_whole.merge(top_all_true, how = 'left', on = ['userID', 'itemID'], suffixes = (None, "_true"))
top_k_predictions_whole["rank_true"] = top_k_predictions_whole["rank_true"].convert_dtypes()
top_k_predictions_whole.rename(columns = {"itemID": "predicted product", "rating": "predicted product's true sales", "rank_true": "predicted product's true rank"}, inplace = True)
# add columns of true top-ranked products
top_all_true.rename(columns = {"itemID": "true product", "rating": "true product's sales"}, inplace = True)
top_k_predictions_whole = top_k_predictions_whole.merge(top_all_true, how = 'left', on = ['userID', 'rank'])
top_k_predictions_whole.rename(columns = {"userID": "location_id"}, inplace = True)
# reorder columns
top_k_predictions_whole = top_k_predictions_whole[["location_id", "rank", "true product", "true product's sales", "predicted product", "predicted product's true sales", "predicted product's true rank"]]
if EXTRA_COLS:
    # add column of fraction of locations selling predicted product
    top_k_predictions_whole["predicted product sells at fraction of locations"] = top_k_predictions_whole.apply(lambda row: round(n_locs_w_prod[row["predicted product"]] / n_locs, 2), axis = 1)
    # add column of average rank of predicted product at locations selling it
    prod_avg_rank = {prod: round(top_all_true[top_all_true["true product"] == prod]["rank"].mean()) for prod in n_locs_w_prod}
    top_k_predictions_whole["predicted product's average rank at locations selling it"] = top_k_predictions_whole.apply(lambda row: prod_avg_rank[row["predicted product"]], axis = 1)
    # save to csv
    top_k_predictions_whole.to_csv(THIS_ENGINE_NAME + "_" + COUNTRY + "_top_" + str(TOP_K_WHOLE) + "_prod_recs_extra_cols.csv")
else:
    top_k_predictions_whole.to_csv(THIS_ENGINE_NAME + "_" + COUNTRY + "_top_" + str(TOP_K_WHOLE) + "_prod_recs.csv")

/Users/DanielShen/opt/anaconda3/envs/microsoft_rec/lib/python3.8/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)1.2s: train loss = 0.62339 = (mf)0.62332 + (embed)0.00007
Epoch 2 (train)1.1s: train loss = 0.38218 = (mf)0.38179 + (embed)0.00039
Epoch 3 (train)1.0s: train loss = 0.33754 = (mf)0.33695 + (embed)0.00059
Epoch 4 (train)0.9s: train loss = 0.31578 = (mf)0.31511 + (embed)0.00067
Epoch 5 (train)0.8s + (eval)0.3s: train loss = 0.28610 = (mf)0.28532 + (embed)0.00078, recall = 0.43669, ndcg = 0.31462, precision = 0.13217, map = 0.11405
Epoch 6 (train)0.8s: train loss = 0.26641 = (mf)0.26550 + (embed)0.00092
Epoch 7 (train)0.8s: train loss = 0.25709 = (mf)0.25605 + (embed)0.00104
Epoch 8 (train)0.8s: train loss = 0.24585 = (mf)0.24469 + (embed)0.00116
Epoch 9 (train)0.8s: train loss = 0.23449 = (mf)0.23321 + (embed)0.00128
Epoch 10 (train)0.8s + (eval)0.2s: train loss = 0.22897 = (mf)0.22759 + (embed)0.00139, recall = 0.46592, ndcg = 0.33512, precision = 0.14078, map = 0.12640
Epoch